In [8]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Neural_CDE/applications-of-NDE-dev

%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/content/drive/My Drive/Neural_CDE/applications-of-NDE-dev')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/Neural_CDE/applications-of-NDE-dev'
/content/drive/MyDrive/Neural_CDE/applications-of-NDE-dev
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
!pip install gymnasium

#Setting up pygame for colab
!python -m pip install pygame
import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))


# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt


#for neural CDE
!pip install torchcde

import torchcde
import torch

In [11]:
!python main.py

pygame 2.2.0 (SDL 2.0.22, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
Traceback (most recent call last):
  File "main.py", line 15, in <module>
    train_X, train_y = data_gen.get_X_y(policy1,100,20)
  File "/content/drive/MyDrive/Neural_CDE/applications-of-NDE-dev/generator.py", line 49, in get_X_y
    state_time_series = self.get_time_series(policy, n, t)
  File "/content/drive/MyDrive/Neural_CDE/applications-of-NDE-dev/generator.py", line 37, in get_time_series
    observation = dmp.get_obs()
  File "/content/drive/MyDrive/Neural_CDE/applications-of-NDE-dev/MP.py", line 19, in get_obs
    return torch.hstack((self.observation,self.policy.weights))
TypeError: expected Tensor as element 0 in argument 0, but got tuple
